### NWS Forecast Data
The NWS doesn't store historical forecast data, so this notebook will just be for prototyping the scraping functions for the DAG (i.e., no preliminary scrape and data load as with the USCRN data).

In [3]:
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pytz
import requests
import pandas as pd
import numpy as np
import os

But first, let's save the relevant urls for each station location -- this will make it easier when we're retrieving the results for each station as part of our DAG.

In [3]:
def getUrl(row):
  """Construct NWS forecast url from latitude and longitude columns"""
  url = f"https://forecast.weather.gov/MapClick.php?lat={row['latitude']}&lon={row['longitude']}&unit=0&lg=english&FcstType=digital&menu=1"
  return url

locations = pd.read_csv("data/locations.csv")
locations['nws_url'] = locations.apply(getUrl, axis=1)
locations.to_csv("data/locations.csv", index=False)

#### Prototyping 
Let's scrape the tabular data from one of the NWS pages and transform it into a dataframe.

In [4]:
from random import choice

url = requests.get(choice(list(locations['nws_url'])))
soup = BeautifulSoup(url.content, "html.parser")

tables = soup.find_all('table')
data = tables[5].find_all("tr")

# Data tables are divided with colspan elements
colspan = data[0]
indices = [i for i, x in enumerate(data) if x == colspan]

dt1 = data[indices[0]+1:indices[1]] # first 24hrs
dt2 = data[indices[1]+1:] # next 24hrs

# Combining these into one dataframe
def getDF(l1, l2): 

  ls = l1.copy()
  ls.extend(l2)

  data_map = {}

  for ele in ls:
    row = [x.getText() for x in ele.find_all("font")]

    if row[0] not in data_map.keys(): # elements from l1
      data_map[row[0]] = row[1:]
    else: # elements from l2
      data_map[row[0]].extend(row[1:])

  df = pd.DataFrame(data_map)
  df['Date'][df['Date']==""] = np.NaN
  df['Date'].ffill(inplace=True)

  print(sys.getsizeof(df), sys.getsizeof(df.to_dict())) 
  return df

getDF(dt1, dt2).sample(5)

45470 656


,Date,Hour (AKST),Temperature (°F),Dewpoint (°F),Wind Chill (°F),Surface Wind (mph),Wind Dir,Gust,Sky Cover (%),Precipitation Potential (%),Relative Humidity (%),Rain,Thunder,Snow,Freezing Rain,Sleet
26,02/16,18,26,20,20,5,NE,,74,46,79,--,--,Chc,--,--
11,02/16,03,2,-4,2,1,E,,92,37,75,--,--,Chc,--,--
42,02/17,10,23,19,14,7,S,,96,82,85,--,--,Ocnl,--,--
14,02/16,06,2,-4,2,0,SW,,86,45,76,--,--,Chc,--,--
4,02/15,20,9,3,9,1,NW,,34,1,77,--,--,--,--,--


Notice the size penalty you get from representing data as a dataframe versus a simpler data type like a dictionary. We'll keep this in mind when we refactor our code next. 

#### Refactoring

In [1]:
# ## Old Refactor

# def getDict(lists:list):  # Refactoring from getDF to process all the tables, not just two
#   """Unnest list of tabular data and combine into one dictionary""" 
#   ls = list(itertools.chain.from_iterable(lists)) # Unnest lists 
#   # Note: each list in lists needs to have a field inserted for station_location prior to being fed into this function
#   data_map = {}
#   for ele in ls:
#     row = [x.getText() for x in ele.find_all("font")]
#     if row[0] not in data_map.keys(): # elements from l1
#       data_map[row[0]] = row[1:]
#     else: # elements from l2
#       data_map[row[0]].extend(row[1:])

#   data_map['Date'] = ffList(data_map['Date'])

#   return data_map


# def getForecast():
#   """Get data from soup objects created from NWS urls in locations.csv, then write to .csv"""
 
#   locations = pd.read_csv("data/locations.csv")
#   loc_dict = dict(zip(locations['station_location'], locations['nws_url']))
  
#   ls = []
#   for station, url in loc_dict.items():
#     result = requests.get(url)
#     soup = BeautifulSoup(result.content, "html.parser")
#     data = soup.find_all("table")[5].find_all("tr")

#     # 'data' is divided into two tables by two colspan elements
#     colspan = data[0]
#     data = [x for  x in data if x != colspan] # add station_name here 
#     ls.append(data)

#   return getDict(ls)
# # pd.DataFrame(getForecast())  

# pd.DataFrame(getForecast())

##### utils.py

In [1]:
def ffList(ls:list) -> list:
  """Like ffill() from pandas, except for lists"""
  for i in range(len(ls)):
    if not ls[i] and i > 0:
        ls[i] = ls[i-1]
  return ls

def getColsFromTable(table:list, location:str):
  """Get cols from list of <tr> elements"""
  cols = [[ele.getText() for ele in tr.find_all("font")] for tr in table] # these are rows in the table's current landscape orientation
  location_col = ['location']
  location_col.extend([location]*24)
  cols.insert(1, location_col)
  cols.insert(19, location_col) # for second table
  return cols

def getDict(col_list:list):
  """Get dictionary from list of columns (which are also lists)"""
  data_map = {}
  for col in col_list:
    if col[0] not in data_map.keys(): # cols from first half of table
      data_map[col[0]] = col[1:]
    else: # cols from second half
      data_map[col[0]].extend(col[1:])
  data_map['Date'] = ffList(data_map['Date'])
  return data_map

##### nws_dag.py

In [6]:
def getForecast():
  """Get dictionary of forecast data for next 48 hours from various points in Alaska"""
  locations = pd.read_csv("data/locations.csv")
  loc_dict = dict(zip(locations['station_location'], locations['nws_url']))

  col_list = []
  for location, url in loc_dict.items():
    result = requests.get(url)
    soup = BeautifulSoup(result.content, "html.parser")
    table48 = soup.find_all("table")[5].find_all("tr") # list of <tr> elements from main data table (really two tables combined: one for each day in next 48h period)
    colspan = table48[0]  # divided into two tables by two colspan elements
    table48 = [tr for  tr in table48 if tr != colspan] # remove colspan elements

    cols = getColsFromTable(table48,location)    
    col_list.extend(cols)
  
  return getDict(col_list)

def transformDF(myDict): 
  """Cast dictionary from getForecast() to a dataframe, transform, and write (append) to .csv"""
  df = pd.DataFrame(myDict)
  df.columns = [col.lower() for col in df.columns] 
  df.replace({'':np.NaN, '--':np.NaN}, inplace=True)

  ## Datetime Transformations
  cur_year = datetime.now().year
  dt_strings = df['date'] + '/' + str(cur_year) + ' ' + df['hour (akst)'] + ':00 AKST'
  # Local time (AKST)
  df['lst_datetime'] = pd.to_datetime(dt_strings, format='%m/%d/%Y %H:%M AKST')
  # UTC time
  akst_offset = timedelta(hours=9)
  df['utc_datetime'] = df['lst_datetime'] + akst_offset

  # reorder columns 
  cols = ['location','utc_datetime','lst_datetime'] + list(df.columns)[3:-2]
  df = df[cols]

  # timestamp column: track when forecast was accessed -- DAG will run every 48 hours
  df['fcst_date'] = datetime.now()

  ## Write to CSV
  # hdr = False  if os.path.isfile('data/forecasts.csv') else True
  # df.to_csv('data/forecasts.csv', mode='a', index=False, header=hdr)

  return df